In [20]:
# import the inference-sdk
from inference_sdk import InferenceHTTPClient
import cv2
import time

# initialize the client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="99oKvIcFbNcIWjEIglpT"
)

# Path to the video file
video_path = r"C:\Users\prol-\Documents\Masters\Thesis\gopro\test.mp4"  # Replace with your video path

# Open the video file
cap = cv2.VideoCapture(video_path)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {total_frames}")

# Check if the video was opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_skip = 1  # Process every 2nd frame
frame_count = 0

# Loop through each frame of the video
while True:
    start_time = time.time()  # Start timing

    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Skip this frame
    print(frame_count)
    
    # Rotate the frame by 90 degrees clockwise
    #frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

    # Resize the frame to a smaller resolution
    resized_frame = cv2.resize(frame, (320, 180))  # Adjust the size as needed
    
    # Perform inference on the resized frame
    result = CLIENT.infer(resized_frame, model_id="test-tws1v/3")
    
    # Scale the bounding box coordinates back to the original frame size
    scale_x = frame.shape[1] / resized_frame.shape[1]
    scale_y = frame.shape[0] / resized_frame.shape[0]
    
    for prediction in result["predictions"]:
        x = int(prediction["x"] * scale_x)
        y = int(prediction["y"] * scale_y)
        width = int(prediction["width"] * scale_x)
        height = int(prediction["height"] * scale_y)
    
        label = prediction["class"]
        confidence = prediction["confidence"]

        # Convert center coordinates to top-left and bottom-right coordinates
        x1 = int(x - width / 2)
        y1 = int(y - height / 2)
        x2 = int(x + width / 2)
        y2 = int(y + height / 2)

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw label and confidence
        text = f"{label} {confidence:.2f}"
        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


    # Уменьшаем размер кадра перед отображением
    scale_factor = 0.5  # Измени на 0.3 или другое значение, если нужно сильнее уменьшить
    display_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
    
    cv2.imshow('Video', display_frame)

    # Display the annotated frame
    #cv2.imshow('Video', frame)
    

    # Calculate and print the time taken for inference
    end_time = time.time()
    print(f"Time per frame: {end_time - start_time:.2f} seconds")
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

Total frames in video: 9596
1
Time per frame: 1.28 seconds
2
Time per frame: 1.00 seconds
3
Time per frame: 0.51 seconds
4
Time per frame: 0.37 seconds
5
Time per frame: 0.40 seconds
6
Time per frame: 0.40 seconds
7
Time per frame: 1.53 seconds
8
Time per frame: 1.43 seconds
9
Time per frame: 0.40 seconds
10
Time per frame: 0.34 seconds
11
Time per frame: 0.42 seconds
12
Time per frame: 1.82 seconds
13
Time per frame: 1.76 seconds
14
Time per frame: 1.89 seconds
15
Time per frame: 0.48 seconds
16
Time per frame: 0.43 seconds
17
Time per frame: 0.41 seconds
18
Time per frame: 1.25 seconds
19
Time per frame: 0.30 seconds
20
Time per frame: 0.36 seconds
21
Time per frame: 0.40 seconds
22
Time per frame: 0.33 seconds
23
Time per frame: 0.50 seconds
24
Time per frame: 1.43 seconds
25
Time per frame: 0.81 seconds
26
Time per frame: 1.43 seconds
27
Time per frame: 11.98 seconds
28
Time per frame: 0.37 seconds
29
Time per frame: 1.30 seconds
30
Time per frame: 0.96 seconds
31
Time per frame: 0

KeyboardInterrupt: 

In [23]:
cv2.destroyAllWindows()

In [22]:
# import the inference-sdk
from inference_sdk import InferenceHTTPClient
import cv2
import time
import os

# Initialize the client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="99oKvIcFbNcIWjEIglpT"
)

# Path to the video file
video_path = r"C:\Users\prol-\Documents\Masters\Thesis\gopro\test.mp4"  # Replace with your video path

# Create a directory to save bee detections
output_dir = r"C:\Users\prol-\Documents\Masters\Thesis\dataset"
os.makedirs(output_dir, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {total_frames}")

# Check if the video was opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_skip = 1  # Process every frame
frame_count = 0
bee_count = 0  # Counter for bee images

# Loop through each frame of the video
while True:
    start_time = time.time()  # Start timing

    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Skip this frame
    print(f"Processing frame {frame_count}")
    
    # Resize the frame to a smaller resolution for inference
    resized_frame = cv2.resize(frame, (320, 180))  # Adjust the size as needed
    
    # Perform inference on the resized frame
    result = CLIENT.infer(resized_frame, model_id="test-tws1v/3")
    
    # Scale the bounding box coordinates back to the original frame size
    scale_x = frame.shape[1] / resized_frame.shape[1]
    scale_y = frame.shape[0] / resized_frame.shape[0]
    
    for prediction in result["predictions"]:
        label = prediction["class"]
        confidence = prediction["confidence"]

        # Process only bees
        if label.lower() == "bee" and confidence > 0.5:  # Only save high-confidence detections
            x = int(prediction["x"] * scale_x)
            y = int(prediction["y"] * scale_y)
            width = int(prediction["width"] * scale_x)
            height = int(prediction["height"] * scale_y)

            # Convert center coordinates to top-left and bottom-right coordinates
            x1 = max(0, int(x - width / 2))
            y1 = max(0, int(y - height / 2))
            x2 = min(frame.shape[1], int(x + width / 2))
            y2 = min(frame.shape[0], int(y + height / 2))

            # Extract the bounding box region (bee image)
            bee_img = frame[y1:y2, x1:x2]

            # Save the cropped bee image
            bee_filename = os.path.join(output_dir, f"bee_{bee_count:05d}.jpg")
            cv2.imwrite(bee_filename, bee_img)
            bee_count += 1
            print(f"Saved {bee_filename}")

            # Draw bounding box on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw label and confidence
            text = f"{label} {confidence:.2f}"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Reduce frame size before displaying
    scale_factor = 0.5
    display_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
    
    cv2.imshow('Video', display_frame)

    # Calculate and print the time taken for inference
    end_time = time.time()
    print(f"Time per frame: {end_time - start_time:.2f} seconds")
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


Total frames in video: 9596
Processing frame 1
Saved C:\Users\prol-\Documents\Masters\Thesis\dataset\bee_00000.jpg
Time per frame: 2.88 seconds
Processing frame 2
Saved C:\Users\prol-\Documents\Masters\Thesis\dataset\bee_00001.jpg
Time per frame: 1.54 seconds
Processing frame 3
Saved C:\Users\prol-\Documents\Masters\Thesis\dataset\bee_00002.jpg
Time per frame: 1.95 seconds
Processing frame 4
Saved C:\Users\prol-\Documents\Masters\Thesis\dataset\bee_00003.jpg
Time per frame: 1.19 seconds
Processing frame 5
Saved C:\Users\prol-\Documents\Masters\Thesis\dataset\bee_00004.jpg
Time per frame: 2.41 seconds
Processing frame 6
Saved C:\Users\prol-\Documents\Masters\Thesis\dataset\bee_00005.jpg
Time per frame: 0.69 seconds
Processing frame 7
Saved C:\Users\prol-\Documents\Masters\Thesis\dataset\bee_00006.jpg
Time per frame: 0.55 seconds
Processing frame 8
Saved C:\Users\prol-\Documents\Masters\Thesis\dataset\bee_00007.jpg
Time per frame: 1.39 seconds
Processing frame 9
Saved C:\Users\prol-\Docu

KeyboardInterrupt: 